In [1]:
import cv2
import os
import numpy as np
import pycocotools

In [2]:
#xml to dict
#https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=pk3152&logNo=221367256441
import xmltodict
import json

In [11]:
# 압축을 풀고 생기는 최상위 디렉토리
DATA_ROOT = '/Users/sm/Downloads/인도보행 영상/바운딩박스/Bbox_21_new'
# 폴더인 경우만 리스트에 추가
folder_list = [i for i in os.listdir(DATA_ROOT) if os.path.isdir(os.path.join(DATA_ROOT, i))]
print(folder_list)

['Bbox_1671', 'Bbox_1685', 'Bbox_1682', 'Bbox_1676', 'Bbox_1678', 'Bbox_1679', 'Bbox_1677', 'Bbox_1683', 'Bbox_1684', 'Bbox_1702', 'Bbox_1705', 'Bbox_1733', 'Bbox_1734', 'Bbox_1760', 'Bbox_1794', 'Bbox_1758', 'Bbox_1793', 'Bbox_1767', 'Bbox_1751', 'Bbox_1769', 'Bbox_1756', 'Bbox_1735', 'Bbox_1732', 'Bbox_1704', 'Bbox_1703', 'Bbox_1768', 'Bbox_1757', 'Bbox_1750', 'Bbox_1759', 'Bbox_1766', 'Bbox_1792', 'Bbox_1795', 'Bbox_1761', 'Bbox_1743', 'Bbox_1788', 'Bbox_1744', 'Bbox_1772', 'Bbox_1786', 'Bbox_1781', 'Bbox_1775', 'Bbox_1721', 'Bbox_1719', 'Bbox_1726', 'Bbox_1710', 'Bbox_1728', 'Bbox_1717', 'Bbox_1774', 'Bbox_1780', 'Bbox_1787', 'Bbox_1773', 'Bbox_1745', 'Bbox_1742', 'Bbox_1789', 'Bbox_1729', 'Bbox_1716', 'Bbox_1711', 'Bbox_1718', 'Bbox_1727', 'Bbox_1720', 'Bbox_1699', 'Bbox_1807', 'Bbox_1800', 'Bbox_1809', 'Bbox_1697', 'Bbox_1690', 'Bbox_1691', 'Bbox_1696', 'Bbox_1808', 'Bbox_1801', 'Bbox_1806', 'Bbox_1698', 'Bbox_1681', 'Bbox_1675', 'Bbox_1672', 'Bbox_1686', 'Bbox_1688', 'Bbox_1689'

In [19]:
# 폴더에서 xml 확장자를 가진 경우만 리스트업
for folder in folder_list:
    xml_file = [os.path.join(folder,i) for i in os.listdir(os.path.join(DATA_ROOT, folder)) if i.endswith('.xml')]
    print(xml_file)

['Bbox_1671/P1004_17.xml']
['Bbox_1685/P1005_01.xml']
['Bbox_1682/P1004_28.xml']
['Bbox_1676/P1004_22.xml']
['Bbox_1678/P1004_24.xml']
['Bbox_1679/P1004_25.xml']
['Bbox_1677/P1004_23.xml']
['Bbox_1683/P1004_29.xml']
['Bbox_1684/P1004_30.xml']
['Bbox_1702/P1005_18.xml']
['Bbox_1705/P1005_21.xml']
['Bbox_1733/P1007_19.xml']
['Bbox_1734/P1007_20.xml']
['Bbox_1760/P1008_16.xml']
['Bbox_1794/P1009_20.xml']
['Bbox_1758/P1008_14.xml']
['Bbox_1793/P1009_19.xml']
['Bbox_1767/P1008_23.xml']
['Bbox_1751/P1008_07.xml']
['Bbox_1769/P1008_25.xml']
['Bbox_1756/P1008_12.xml']
['Bbox_1735/P1007_21.xml']
['Bbox_1732/P1007_18.xml']
['Bbox_1704/P1005_20.xml']
['Bbox_1703/P1005_19.xml']
['Bbox_1768/P1008_24.xml']
['Bbox_1757/P1008_13.xml']
['Bbox_1750/P1008_06.xml']
['Bbox_1759/P1008_15.xml']
['Bbox_1766/P1008_22.xml']
['Bbox_1792/P1009_18.xml']
['Bbox_1795/P1009_21.xml']
['Bbox_1761/P1008_17.xml']
['Bbox_1743/P1007_29.xml']
['Bbox_1788/P1009_14.xml']
['Bbox_1744/P1007_30.xml']
['Bbox_1772/P1008_28.xml']
[

In [53]:
# xml 파일을 읽어서 dict 로 접근할 수 있게 함
with open(os.path.join(DATA_ROOT, xml_file[0]), 'r') as f:
    clean_xml = xmltodict.parse(f.read())
    # 이렇게 하면 일반적으로 사용하던 json dict 로 편하게 사용할 수 있다고 함
    clean_xml = json.dumps(clean_xml)
    clean_xml = json.loads(clean_xml)

# xml 에 들어있던 기본정보들
print(f'clean_xml.keys() : {clean_xml.keys()}')
print(f"clean_xml['annotations'].keys() : {clean_xml['annotations'].keys()}")
print(f"clean_xml['annotations']['version'] : {clean_xml['annotations']['version']}")
print(f"clean_xml['annotations']['meta'].keys() : {clean_xml['annotations']['meta'].keys()}")
print(f"len(clean_xml['annotations']['image']) : {len(clean_xml['annotations']['image'])}")
# 해당 폴더안에 있는 이미지 갯수랑 실제 xml 에 등록되어 있는 갯수랑 맞는지 확인하기 위함
print(f"image counts : {len([i for i in os.listdir(os.path.join(DATA_ROOT, xml_file[0].split('/')[0])) if i.endswith('.png')])}")

clean_xml.keys() : dict_keys(['annotations'])
clean_xml['annotations'].keys() : dict_keys(['version', 'meta', 'image'])
clean_xml['annotations']['version'] : 1.1
clean_xml['annotations']['meta'].keys() : dict_keys(['task', 'dumped'])
len(clean_xml['annotations']['image']) : 50
image counts : 50


In [60]:
# 이미지를 위한 정보를 찾아보자
# 가장 첫번째 이미지인 0번째 이미지부터 시작
print(clean_xml['annotations']['image'][0].keys())
target_list = ('car', 'person', 'traffic_light', 'truck', 'bicycle', 'motorcycle', 'bus', 'carrier', 'stroller', 'scooter', 'wheelchair')


for k,v in clean_xml['annotations']['image'][0].items():
    print(f'key : {k}, data_type : {type(v)}')
#box 가 list type 이니까 몇 개인지 파악해보자
print()
print(f"len(clean_xml['annotations']['image'][0]['box']) : {len(clean_xml['annotations']['image'][0]['box'])}")
print(f"box keys : {clean_xml['annotations']['image'][0]['box'][0].keys()}")
print('-'*30)
# 생각해보니까 list 여서 del 을 쓰는게 아니다
# list 는 잘못 삭제하면 for loop 이 의도하지 않은대로 동작할 수 있어서 조심해야한다.
for item in clean_xml['annotations']['image'][0]['box']:
    if item['@label'] not in target_list:
        print(f"del {item['@label']} : {item}")
        del(item) # 이 부분 고민해야함
print('-'*30)
print(f"after len(clean_xml['annotations']['image'][0]['box']) : {len(clean_xml['annotations']['image'][0]['box'])}")
print(clean_xml['annotations']['image'][0]['box'])

dict_keys(['@id', '@height', '@width', '@name', 'box'])
key : @id, data_type : <class 'str'>
key : @height, data_type : <class 'str'>
key : @width, data_type : <class 'str'>
key : @name, data_type : <class 'str'>
key : box, data_type : <class 'list'>

len(clean_xml['annotations']['image'][0]['box']) : 9
box keys : dict_keys(['@occluded', '@ybr', '@xbr', '@ytl', '@xtl', '@label'])
------------------------------
del traffic_sign : {'@occluded': '0', '@ybr': '413.19', '@xbr': '633.93', '@ytl': '374', '@xtl': '600.15', '@label': 'traffic_sign'}
del pole : {'@occluded': '1', '@ybr': '569.1', '@xbr': '518.5', '@ytl': '0', '@xtl': '477.7', '@label': 'pole'}
del tree_trunk : {'@occluded': '0', '@ybr': '574.2', '@xbr': '493.2', '@ytl': '259.5', '@xtl': '441.9', '@label': 'tree_trunk'}
del traffic_sign : {'@occluded': '0', '@ybr': '374.03', '@xbr': '633.5', '@ytl': '331.39', '@xtl': '599.73', '@label': 'traffic_sign'}
------------------------------
after len(clean_xml['annotations']['image'][0][

## 앞으로 해야 할 것
- 원하는 리스트의 아이템만 남겨서 일단 저장
- COCO Format 에 필요한 정보들을 가져와서 json type 으로 만들 것